**背景**：序列化的数据（语音，自然语言）

### RNN：

![RNN](RNN.PNG)

### LSTM：

![LSTM](LSTM.PNG)

考虑图片 0~9 数据，图片的每行数据为一个单元，一张图片有28行数据，则图片为步长为28的序列

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from keras import backend as K
K.clear_session()          # 清除之前会话，防止jupyter缓存影响

# 数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 超参数（hyperparameters）
lr = 0.001                    # 学习率
training_iters = 100000       # 循环次数
batch_size = 128              # 定义batch大小

n_inputs = 28                 # 图片 28*28，每行28
n_steps = 28                   # 总行数28
n_hidden_unis = 128           # 隐层神经元（neurons in hidden layer）
n_classes = 10                # 分类 0~9

# 输入
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

# 定义权重和偏差
weights = {
    # （28， 128）
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_unis])),
    # （128， 10）
    'out': tf.Variable(tf.random_normal([n_hidden_unis, n_classes]))
    }
biases = {
    # （128，）
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_unis,])),
    # （10，）
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes,]))
    }

def RNN(X, weights, biases):
    # hidden layer for input to cell
    X =  tf.reshape(X, [-1, n_inputs])                     # X(128 batch, 28 steps, 28 inputs) ==> (128*28, 28 inputs)
    X_in = tf.matmul(X, weights['in'])+biases['in']        # ==> X_in(128*28, 128 hidden)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_unis])  # ==> X_in(128 batch, 28 steps, 128 hidden)
    
    # cell
    # 初始forget_bias为1.0，不forget前面；state_is_tuple=True表示生成的state为元组
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_unis, forget_bias=1.0, state_is_tuple=True)  
    # state：每一步计算的结果，lstm cell分为主线state和分线state：(c_state, m_state)
    _init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    # ime_major=False表示时间点不在维度为1的地方，本例中在X_in的次要维度，即28 steps，第2个维度
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=_init_state, time_major=False)    # 运算
    
    # hidden layer for output as the final results
    # results = tf.matmul(states[1], weights['out']+biases['out'])
    # or
    # 本例中states[1]=output[-1]，经过以下转变后，outputs中有28个元素（即28个步长，图片数据有28行），每个元素为[batch, outputs]
    outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))    # unpack to list[(batch, outputs)..] * steps
    # outputs[-1]最后一个元素，即最后一个步长的计算的结果为最终结果
    results = tf.matmul(outputs[-1], weights['out']+biases['out'])
    
    return results


pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) 
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step*batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={x: batch_xs, y: batch_ys})
        if step%20 == 0:
            print(sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys}))
        step += 1

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
0.125
0.5078125
0.7265625
0.7890625
0.875
0.890625
0.8984375
0.8828125
0.84375
0.921875
0.8671875
0.9375
0.8984375
0.8828125
0.9375
0.96875
0.953125
0.953125
0.9453125
0.953125
0.9140625
0.9453125
0.921875
0.9296875
0.9140625
0.9765625
0.96875
0.953125
0.984375
0.9609375
0.96875
0.9765625
0.9375
0.9765625
0.9609375
0.9609375
0.984375
0.953125
0.9765625
0.9765625


### tf.cast()
将张量转换为一种新的数据类型

### tf.equal(A, B)
对比 A，B 这两个矩阵或者向量的元素，若相等则返回True，否则返回False

In [3]:
import tensorflow as tf

X = [1,2,3,4,5,6]
Y = [1,2,3,7,8,9]

with tf.Session() as sess:
    print(sess.run(tf.equal(X, Y)))

[ True  True  True False False False]


### tf.argmax(X, axis)
根据axis返回该维度的最大值的索引

**示例**：
1. axis=0，返回列最大值索引

In [7]:
X = [[1,2,3],
     [3,2,1]
    ]

with tf.Session() as sess:
    print(sess.run(tf.argmax(X, 0)))

[1 0 0]


2. axis=1，返回行最大值索引

In [8]:
X = [[1,2,3],
     [3,2,1]
    ]

with tf.Session() as sess:
    print(sess.run(tf.argmax(X, 1)))

[2 0]


### shape与reshape

In [16]:
import numpy as np

X = np.array([[[1,2,3,4],
               [5,4,3,8],
               [9,10,11,12]],
     
              [[13,14,15,16],
               [17,18,19,20],
               [21,22,23,24]]
])
X.shape

(2, 3, 4)

In [19]:
Y = X.reshape([-1,4])
Y

array([[ 1,  2,  3,  4],
       [ 5,  4,  3,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16],
       [17, 18, 19, 20],
       [21, 22, 23, 24]])

### tf.transpose(X, perm=None, name='transpose', conjugate=False)
* X：需要变换的张量
* perm：新的维度序列
* name：（可选）操作名称
* conjugate：（可选），设置为True，等价于tf.conj(tf.transpose())


1. 二维

In [26]:
import tensorflow as tf
import numpy as np
 
A = np.array([[1, 2, 3], [4, 5, 6]])
X = tf.transpose(A, [1, 0])             # 相当于转置
 
with tf.Session() as sess:
    print('original:\n',A)
    print('tranpose:\n',sess.run(X))

original:
 [[1 2 3]
 [4 5 6]]
tranpose:
 [[1 4]
 [2 5]
 [3 6]]


2. 三维

In [38]:
import tensorflow as tf
import numpy as np
 
A = np.arange(12).reshape([2,3,2])
X = tf.transpose(A,[0,2,1])
Y = tf.transpose(A,[1,0,2])
with tf.Session() as sess:
    print('original:\n', A)
    print('A.shape：', A.shape)
    print('='*30)
    
    print('transpose [0,2,1]:\n', sess.run(X))
    print('X.shape：', X.shape)
    print('='*30)
    
    print('transpose [1,0,2]:\n', sess.run(Y))
    print('Y.shape：', Y.shape)

original:
 [[[ 0  1]
  [ 2  3]
  [ 4  5]]

 [[ 6  7]
  [ 8  9]
  [10 11]]]
A.shape： (2, 3, 2)
transpose [0,2,1]:
 [[[ 0  2  4]
  [ 1  3  5]]

 [[ 6  8 10]
  [ 7  9 11]]]
X.shape： (2, 2, 3)
transpose [1,0,2]:
 [[[ 0  1]
  [ 6  7]]

 [[ 2  3]
  [ 8  9]]

 [[ 4  5]
  [10 11]]]
Y.shape： (3, 2, 2)


### X.shape 和 X.get_shape()
* 获取张量的shape

### 可采用 as_list() 转换为 list
* X.shape.as_list()
* X.get_shape().as_list()

### tf.unstack(A, axis=0)
* 拆分矩阵
* axis=0，则按行拆分；axis=1，则按列拆分

In [55]:
import tensorflow as tf
 
A = [[1, 2, 3], 
     [4, 5, 6]]
X = tf.unstack(A, axis=0)
 
with tf.Session() as sess:
    print(sess.run(X))

[array([1, 2, 3]), array([4, 5, 6])]


In [53]:
import tensorflow as tf
 
A = [[1, 2, 3], 
     [4, 5, 6]]
X = tf.unstack(A, axis=1)
 
with tf.Session() as sess:
    print(sess.run(X))

[array([1, 4]), array([2, 5]), array([3, 6])]


### tf.stack(A, axis=0)
* 矩阵拼接
* axis=0，则在维度0上拼接（第一个维度）；axis=1，则在维度1上拼接（第二个维度）

In [62]:
import tensorflow as tf
 
A = [[1, 2, 3], 
     [4, 5, 6]]
B = [[10, 20, 30], 
     [40, 50, 60]]
X = tf.stack([A, B], axis=0)    # 0维拼接
 
with tf.Session() as sess:
    print(sess.run(X))

[[[ 1  2  3]
  [ 4  5  6]]

 [[10 20 30]
  [40 50 60]]]


In [59]:
import tensorflow as tf
 
A = [[1, 2, 3], 
     [4, 5, 6]]
B = [[10, 20, 30], 
     [40, 50, 60]]
X = tf.stack([A, B], axis=1)   # 1维拼接
 
with tf.Session() as sess:
    print(sess.run(X))

[[[ 1  2  3]
  [10 20 30]]

 [[ 4  5  6]
  [40 50 60]]]


In [60]:
import tensorflow as tf
 
A = [[1, 2, 3], 
     [4, 5, 6]]
B = [[10, 20, 30], 
     [40, 50, 60]]
X = tf.stack([A, B], axis=2)   # 1维拼接
 
with tf.Session() as sess:
    print(sess.run(X))

[[[ 1 10]
  [ 2 20]
  [ 3 30]]

 [[ 4 40]
  [ 5 50]
  [ 6 60]]]
